# Advanced CMB Analysis Techniques

This notebook demonstrates advanced analysis techniques including:
- Custom model implementation
- Advanced MCMC techniques
- Error analysis
- Publication-quality visualization

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cmb_analysis.cosmology import CosmologyModel, LCDM
from cmb_analysis.analysis import (
    PowerSpectrumCalculator,
    MCMCAnalysis,
    CosmicTransferFunctions
)
from cmb_analysis.visualization import CMBPlotter, MCMCDiagnostics
from cmb_analysis.utils.statistics import fisher_matrix

%matplotlib inline
plt.style.use('seaborn-v0_8-paper')

## 1. Implementing a Custom Cosmological Model

Let's implement a modified gravity model as an example.

In [ ]:
class ModifiedGravityModel(CosmologyModel):
    """Example modified gravity model with additional parameter μ_0."""
    
    def __init__(self):
        super().__init__()
        self.param_info = {
            'H0': (67.32, 0.54),
            'omega_b': (0.02237, 0.00015),
            'omega_cdm': (0.1200, 0.0012),
            'tau': (0.0544, 0.0073),
            'ns': (0.9649, 0.0042),
            'ln10As': (3.044, 0.014),
            'mu_0': (1.0, 0.1)  # Modified gravity parameter
        }
    
    def H(self, z, params):
        """Modified Hubble parameter."""
        H0 = params['H0']
        omega_m = (params['omega_b'] + params['omega_cdm']) / (H0/100)**2
        mu = params['mu_0'] * (1 + z)**(-0.5)  # Scale-dependent modification
        return H0 * np.sqrt(omega_m * (1+z)**3 * mu + (1-omega_m))

# Initialize model
mg_model = ModifiedGravityModel()

## 2. Advanced MCMC Techniques

Let's implement parallel tempering MCMC for better exploration of parameter space.

In [ ]:
def setup_parallel_tempering(n_temps=5, max_temp=50):
    """Set up parallel tempering temperatures."""
    return np.geomspace(1, max_temp, n_temps)

# Generate mock data
calculator = PowerSpectrumCalculator()
true_params = mg_model.param_info.copy()
true_params['mu_0'] = (1.0, 0.1)  # Add modified gravity parameter

# Set up parallel tempering
temperatures = setup_parallel_tempering()
chains = []

for temp in temperatures:
    mcmc = MCMCAnalysis(
        calculator,
        mock_data,
        true_params,
        temperature=temp
    )
    mcmc.run_mcmc()
    chains.append(mcmc)

## 3. Advanced Error Analysis

Let's compute Fisher matrices and perform error forecasting.

In [ ]:
# Compute Fisher matrix
def compute_fisher(params, calculator, errors):
    return fisher_matrix(
        params,
        calculator.compute_all_spectra,
        errors
    )

# Get Fisher matrix
F = compute_fisher(true_params, calculator, mock_data)

# Compute parameter uncertainties
param_errors = np.sqrt(np.diag(np.linalg.inv(F)))

print("Forecasted parameter uncertainties:")
for param, error in zip(true_params.keys(), param_errors):
    print(f"{param}: ±{error:.6f}")

## 4. Publication-Quality Visualization

Create publication-ready plots with error estimates.

In [ ]:
plotter = CMBPlotter()

# Plot power spectra with theory variations
fig = plotter.plot_theory_variation(
    np.arange(len(mock_data['tt_data'])),
    chains[0].get_chain(flat=True),
    calculator.compute_all_spectra,
    n_curves=100
)

# Save in publication format
plotter.save_publication_plots('power_spectra', fig)

# Create comprehensive diagnostic plot
diagnostics = MCMCDiagnostics()
fig = diagnostics.plot_diagnostic_summary(
    chains[0].get_chain(),
    list(true_params.keys()),
    chains[0].sampler.acceptance_fraction.mean()
)
plt.show()

## 5. Model Comparison

Compare standard ΛCDM with our modified gravity model.

In [ ]:
def compute_evidence(chain):
    """Compute model evidence using thermodynamic integration."""
    log_likes = chain.get_log_prob(flat=True)
    return np.mean(log_likes)

# Compare models
lcdm_model = LCDM()
lcdm_chain = MCMCAnalysis(calculator, mock_data, lcdm_model.param_info)
lcdm_chain.run_mcmc()

# Compute Bayes factor
evidence_mg = compute_evidence(chains[0])
evidence_lcdm = compute_evidence(lcdm_chain)
bayes_factor = np.exp(evidence_mg - evidence_lcdm)

print(f"log Bayes factor (MG vs ΛCDM): {np.log(bayes_factor):.2f}")

## 6. Conclusion

This notebook demonstrated advanced analysis techniques including:
1. Custom model implementation
2. Parallel tempering MCMC
3. Fisher matrix analysis
4. Model comparison
5. Publication-quality visualization

These techniques provide a comprehensive framework for advanced CMB analysis.